In [2]:
import cv2
import numpy as np
from PIL import Image
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing import image
import time
import serial

img_height, img_width = 224, 224

# Function to preprocess the image for prediction
def preprocess_image(img_array):
    img = Image.fromarray(img_array)
    img = img.resize((img_height, img_width))
    img_array = image.img_to_array(img)  # Use the renamed module
    img_array = np.expand_dims(img_array, axis=0)
    return img_array / 255.0

In [3]:
def ardconnect():
    ser = serial.Serial('COM5', 9600) # Open a serial connection to the Arduino
    if ser.isOpen():
        print("Connected to COM5")
        time.sleep(5)
    return ser

def send_command(ser, predicted_class): # Function to send commands to Arduino
    if predicted_class == 3:
        print("sending command A")
        ser.write(b'A')

    if predicted_class == 1 or predicted_class == 4:
        print("sending command B")
        ser.write(b'B')

    if predicted_class == 2 or predicted_class == 5:
        print("sending command C")
        ser.write(b'C')

def ardClose(ser):
    ser.close()
    print("COM5 port closed")

In [4]:
# Reading the input using the camera
def read_cam(cam):
    result, img_from_camera = cam.read()
    return result, img_from_camera

In [5]:
def assign_cat(model, img_from_camera, result):
    # If image is detected without any error, show the result
    if result:
        # Showing result, it takes frame name and image output
        #cv2.imshow("Waste", img_from_camera)

        # Save the image in local storage
        cv2.imwrite("Last Saved.png", img_from_camera)

        # Preprocess the captured image
        your_image_data = np.array(img_from_camera)
        input_data = preprocess_image(your_image_data)

        # Make predictions
        predictions = model.predict(input_data)
        predicted_class = np.argmax(predictions)

        # Print the predicted class
        print(f"Predicted Class: {predicted_class}")
        
        return predicted_class

        # If keyboard interrupt occurs, destroy the image window
        #cv2.destroyWindow("Waste Segregation")

    # If captured image is corrupted or no image is detected, print an error message
    else:
        print("No image detected. Please try again")

In [6]:
def main():
    cam_port = 1
    cam = cv2.VideoCapture(cam_port)

    # Load the trained model
    model_path = 'C:\\Users\\prath\\Downloads\\Garbage_Type_Detection.h5'
    model = load_model(model_path)

    ser = ardconnect()
    
    try:
        while True:
            start_time = time.time()

            # Read the camera every 10 seconds
            while time.time() - start_time < 20:
                result, img_from_camera = read_cam(cam)
                time.sleep(0.1)  # Optional: Adjust sleep time to control the frame rate

            predicted_class = assign_cat(model, img_from_camera, result)
            send_command(ser, predicted_class)

            # Break the loop if the user presses Enter
            if cv2.waitKey(1) == 13:  # 13 is the ASCII code for Enter
                break

        # Release the camera
        cam.release()
        cv2.destroyAllWindows()
        ardClose(ser)
        
    except KeyboardInterrupt:
        pass

# Call the main function
main()



Connected to COM5
1/1 [==============================] - 0s 153ms/step
Predicted Class: 2
sending command C
1/1 [==============================] - 0s 52ms/step
Predicted Class: 2
sending command C
1/1 [==============================] - 0s 54ms/step
Predicted Class: 1
sending command B
1/1 [==============================] - 0s 70ms/step
Predicted Class: 0
1/1 [==============================] - 0s 60ms/step
Predicted Class: 0
1/1 [==============================] - 0s 63ms/step
Predicted Class: 0
1/1 [==============================] - 0s 70ms/step
Predicted Class: 0
1/1 [==============================] - 0s 61ms/step
Predicted Class: 0
1/1 [==============================] - 0s 60ms/step
Predicted Class: 0
1/1 [==============================] - 0s 51ms/step
Predicted Class: 0
1/1 [==============================] - 0s 57ms/step
Predicted Class: 0
1/1 [==============================] - 0s 48ms/step
Predicted Class: 0
1/1 [==============================] - 0s 53ms/step
Predicted Class: 0
1/